In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
country_of_birth_data_d="/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/country_of_birth"
coding_tsv=file.path(country_of_birth_data_d, "misc/ukb2005693_ukb37855.country_of_birth.coding.tsv")
freq_data_dir='private_data'


In [7]:
out_f <- '../UKB.HLA.country_of_birth.tsv'

In [3]:
source('../HLA_functions.R')


In [4]:
coding_df <- fread(coding_tsv, sep='\t') %>%
rename('field_val' = '#field_val')


In [5]:
gcount_df <- coding_df %>%
filter(total_n >= 30) %>%
pull(field_val) %>%
read_plink_out('gcount') %>%
left_join(
    coding_df %>%
    select(field_val, meaning), 
    by='field_val'
) %>%
rename(
    'country_of_birth'='meaning', 
    'country_of_birth_code'='field_val'
)


In [6]:
df <- gcount_df %>% 
select(-CHROM, -HAP_REF_CT, -HAP_ALT_CTS) %>%
mutate(
    Count = HET_REF_ALT_CTS + TWO_ALT_GENO_CTS * 2,
    Frequency = Count / (2 * (HOM_REF_CT + HET_REF_ALT_CTS + TWO_ALT_GENO_CTS)),
) %>%
UKB_HLA_ID_to_4digit_allele() %>%
select(Locus, Allele, country_of_birth, Frequency, Count)


In [8]:
df %>%
rename('#Locus' = 'Locus') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)
